In [11]:
#Data maipulation
import pandas as pd
import numpy as np
import warnings



#Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pickle

#Data modelization
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from category_encoders import TargetEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report,confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc



import scipy.stats as st  # for continuous distributions in randomized search

# 📌 2. Loading Dataset

In [12]:
#importing my pre-cleaned DF
df=pd.read_pickle("/home/utilisateur/Documents/Loan_Projet_Brief/Brief_Loan_Project/data/exp_data.pkl")
df.head()

,State,Zip,BankState,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,MIS_Status,GrAppv,ApprovalMonth,NAICS_CODE
0,IN,47711,OH,1997,84,4,1.0,0,0,0,0,0.0,1.0,60000.0,0,60000.0,2,45
1,IN,46526,IN,1997,60,2,1.0,0,0,0,0,0.0,1.0,40000.0,0,40000.0,2,72
2,IN,47401,IN,1997,180,7,0.0,0,0,0,0,0.0,0.0,287000.0,0,287000.0,2,62
3,OK,74012,OK,1997,60,2,0.0,0,0,0,0,0.0,1.0,35000.0,0,35000.0,2,0
4,FL,32801,FL,1997,240,14,0.0,7,7,0,0,0.0,0.0,229000.0,0,229000.0,2,0


In [13]:
df.columns.tolist()

['State',
 'Zip',
 'BankState',
 'ApprovalFY',
 'Term',
 'NoEmp',
 'NewExist',
 'CreateJob',
 'RetainedJob',
 'FranchiseCode',
 'UrbanRural',
 'RevLineCr',
 'LowDoc',
 'DisbursementGross',
 'MIS_Status',
 'GrAppv',
 'ApprovalMonth',
 'NAICS_CODE']

In [14]:
# 2. Define a helper function to convert binary columns
def convert_binary(X):
    X = X.copy()
    binary_cols = ["RevLineCr", "LowDoc", "NewExist"]
    for col in binary_cols:
        if col in X.columns:
            X[col] = X[col].astype(int)
    return X

In [15]:
# 1. Data preparation
# (Assume df is your preloaded DataFrame)
target = "MIS_Status"
X = df.drop(columns=[target])
y = df[target]

# Columns to impute (example columns)
cols_to_impute = ['State','RevLineCr', 'LowDoc', 'BankState', 'NewExist']
imputer = SimpleImputer(strategy='most_frequent')
X[cols_to_impute] = imputer.fit_transform(X[cols_to_impute])
# Assuming X is your feature DataFrame
X = convert_binary(X)

# Then split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, stratify=y, random_state=42
)

# Identify categorical columns (for target encoding)
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()

In [16]:
#Creating the preprocessor 
warnings.filterwarnings("ignore")


# Define a base pipeline (include additional steps if needed)
base_pipeline = Pipeline([
    ("target_encoder", TargetEncoder(cols=cat_cols, smoothing=0.3)),
    ("classifier", GradientBoostingClassifier(random_state=42))  # placeholder
])

# Define your models
models = {
    #"RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    #"XGBoost": XGBClassifier(eval_metric="logloss", random_state=42)
}

# Define a modified hyperparameter search space with narrower ranges
param_grid = {
    

    "GradientBoosting": {'classifier__learning_rate': [0.0831993941811405], 'classifier__max_depth': [7], 'classifier__n_estimators': [100]},
    
}

results_grid = {}  # Dictionary to store grid search results

for model_name, model in models.items():
    print(f"🚀 Running GridSearchCV for {model_name} ...")
    
    # Update the pipeline to use the current model
    pipeline = Pipeline(base_pipeline.steps[:-1] + [("classifier", model)])
    
    # Setup GridSearchCV
    gs = GridSearchCV(
        pipeline,
        param_grid=param_grid[model_name],
        cv=5,  # you can adjust folds if needed
        scoring="roc_auc",
        n_jobs=-1,
        verbose=1
    )
    
    # Fit GridSearchCV on the training data
    gs.fit(X_train, y_train)

    # Print and store the best parameters
    print(f"Best parameters from GriddSearch for {model_name}:")
    print(gs.best_params_)
    
    # Get the best estimator
    best_model = gs.best_estimator_
    
    # Evaluate on the test set
    y_pred = best_model.predict(X_test)
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred_proba)
    report = classification_report(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    
    # Store the results
    results_grid[model_name] = {
        "Best Grid Model": best_model,
        "Best Grid Params": gs.best_params_,
        "AUC": auc,
        "Report": report,
        "Confusion Matrix": cm
    }
    
    print(f"\n🔹 {model_name} - AUC Score: {auc:.4f}")
    print(report)
    print("Confusion Matrix:")
    print(cm)
    print("="*50, "\n")



🚀 Running GridSearchCV for GradientBoosting ...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters from GriddSearch for GradientBoosting:
{'classifier__learning_rate': 0.0831993941811405, 'classifier__max_depth': 7, 'classifier__n_estimators': 100}

🔹 GradientBoosting - AUC Score: 0.9781
              precision    recall  f1-score   support

           0       0.96      0.98      0.97     74135
           1       0.88      0.82      0.85     15782

    accuracy                           0.95     89917
   macro avg       0.92      0.90      0.91     89917
weighted avg       0.95      0.95      0.95     89917

Confusion Matrix:
[[72368  1767]
 [ 2865 12917]]



In [17]:


# Define the export directory.
export_dir = '/home/utilisateur/Documents/Loan_Projet_Brief/Brief_Loan_Project/src/models/'

# Ensure the directory exists; if not, create it.
os.makedirs(export_dir, exist_ok=True)

# Assuming your grid search results are stored in the results_grid dictionary
# with keys like "RandomForest", "GradientBoosting", and "XGBoost".
for model_name, result in results_grid.items():
    best_model = result["Best Grid Model"]
    file_name = f"{model_name}_best_grid_model.pkl"
    export_path = os.path.join(export_dir, file_name)
    
    with open(export_path, "wb") as f:
        pickle.dump(best_model, f)
    
    print(f"Exported {model_name} best model to {export_path}")


Exported GradientBoosting best model to /home/utilisateur/Documents/Loan_Projet_Brief/Brief_Loan_Project/src/models/GradientBoosting_best_grid_model.pkl
